In [1]:
import json
import numpy as np
import pandas as pd


pd.set_option("display.max_columns", 32)

key = 4
key_size = 32

In [2]:
message_df = pd.read_csv("message.csv", header=None)

message_bk = message_df.to_numpy()

display(message_df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,8,13,60,19,69,92,43,46,87,52,29,42,83,87,89,114,44,21,48,83,96,9,97
1,103,74,116,79,89,56,6,63,85,65,24,86,13,10,43,33,103,87,81,58,109,99,60
2,19,102,95,40,32,26,44,56,44,26,56,91,28,16,28,57,27,18,94,54,19,48,88
3,109,87,0,0,83,73,38,106,101,109,75,13,37,77,32,34,4,6,52,50,104,89,31
4,25,7,1,111,64,45,58,76,13,85,47,115,1,74,76,83,54,18,116,113,56,84,34
5,13,2,34,88,8,102,34,47,51,36,114,69,2,67,73,41,13,97,102,98,113,94,63
6,92,94,65,102,58,55,93,3,16,11,99,80,109,83,98,60,92,25,17,88,31,26,81
7,12,49,86,21,81,75,51,38,35,52,1,86,109,6,66,96,18,70,80,16,65,27,104
8,65,104,78,73,24,64,9,54,80,39,115,116,0,75,21,45,79,102,112,70,19,22,23
9,26,71,100,4,5,102,114,33,24,113,63,91,0,16,7,6,60,62,68,31,85,70,78


## 3.1) Matriz de encriptar utilizada

In [3]:
keys_df = pd.read_csv("keys.csv", header=None)

keys_ls = [
    keys_df.iloc[i : i + key_size].reset_index(drop=True)
    for i in range(0, len(keys_df), key_size)
]

display(keys_ls[key])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,1,1,1,2,1,1,1,2,1,3,1,3,1,5,1,6,1,1,1,2,1,1,1,2,1,3,1,3,1,5,1,6
1,1,2,1,3,1,2,1,3,1,4,1,4,1,6,1,7,1,2,1,3,1,2,1,3,1,4,1,4,1,6,1,7
2,0,0,1,2,0,0,1,2,0,0,1,4,0,0,1,6,0,0,1,2,0,0,1,2,0,0,1,4,0,0,1,6
3,0,0,1,3,0,0,1,3,0,0,1,5,0,0,1,7,0,0,1,3,0,0,1,3,0,0,1,5,0,0,1,7
4,0,0,0,0,1,7,1,7,0,0,0,0,1,5,1,6,0,0,0,0,1,7,1,7,0,0,0,0,1,5,1,6
5,0,0,0,0,1,8,1,8,0,0,0,0,1,6,1,7,0,0,0,0,1,8,1,8,0,0,0,0,1,6,1,7
6,0,0,0,0,0,0,1,8,0,0,0,0,0,0,1,6,0,0,0,0,0,0,1,8,0,0,0,0,0,0,1,6
7,0,0,0,0,0,0,1,9,0,0,0,0,0,0,1,7,0,0,0,0,0,0,1,9,0,0,0,0,0,0,1,7
8,0,0,0,0,0,0,0,0,1,3,1,3,1,5,1,6,0,0,0,0,0,0,0,0,1,3,1,3,1,5,1,6
9,0,0,0,0,0,0,0,0,1,4,1,4,1,6,1,7,0,0,0,0,0,0,0,0,1,4,1,4,1,6,1,7


In [4]:
key_bk = keys_ls[key].to_numpy()

key_bk_inv = np.linalg.inv(key_bk).round(0).astype(int)

display(pd.DataFrame(key_bk_inv))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,2,-1,-2,1,-8,7,9,-8,-2,1,2,-1,8,-7,-9,8,-8,7,9,-8,8,-7,-9,8,1,-3,-1,3,-1,5,1,-6
1,-1,1,1,-1,1,-1,-1,1,1,-1,-1,1,-1,1,1,-1,1,-1,-1,1,-1,1,1,-1,-1,4,1,-4,1,-6,-1,7
2,0,0,3,-2,0,0,-9,8,0,0,-3,2,0,0,9,-8,0,0,-9,8,0,0,9,-8,0,0,1,-4,0,0,-1,6
3,0,0,-1,1,0,0,1,-1,0,0,1,-1,0,0,-1,1,0,0,1,-1,0,0,-1,1,0,0,-1,5,0,0,1,-7
4,0,0,0,0,8,-7,-9,8,0,0,0,0,-8,7,9,-8,0,0,0,0,-1,1,1,-1,0,0,0,0,1,-5,-1,6
5,0,0,0,0,-1,1,1,-1,0,0,0,0,1,-1,-1,1,0,0,0,0,1,-2,-1,2,0,0,0,0,-1,6,1,-7
6,0,0,0,0,0,0,9,-8,0,0,0,0,0,0,-9,8,0,0,0,0,0,0,-1,2,0,0,0,0,0,0,1,-6
7,0,0,0,0,0,0,-1,1,0,0,0,0,0,0,1,-1,0,0,0,0,0,0,1,-3,0,0,0,0,0,0,-1,7
8,0,0,0,0,0,0,0,0,4,-3,-5,4,-4,3,5,-4,0,0,0,0,0,0,0,0,-1,3,1,-3,1,-5,-1,6
9,0,0,0,0,0,0,0,0,-1,1,1,-1,1,-1,-1,1,0,0,0,0,0,0,0,0,1,-4,-1,4,-1,6,1,-7


In [5]:
key_inv_check = np.matmul(key_bk, key_bk_inv)

display(pd.DataFrame(key_inv_check))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
decoded_bk = np.matmul(key_bk_inv, message_bk)

display(pd.DataFrame(decoded_bk))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,1699,-127,15,56,-1279,139,-668,-1770,-100,-546,-694,-1887,249,-417,1303,-1651,-817,1,-1887,-686,-1515,252,354
1,-207,236,118,139,236,-113,137,245,-83,0,240,489,-80,119,-99,240,358,127,361,359,13,-100,-354
2,-2124,760,368,-353,16,0,708,1784,724,237,133,375,-99,-690,-692,1535,355,260,241,-336,827,-345,472
3,373,-235,-117,14,119,14,-233,-109,18,17,-11,5,9,119,136,-118,13,30,13,-98,137,132,0
4,-1411,-217,-236,-334,608,-88,605,839,127,-97,-1180,720,-115,-472,-231,1666,-231,-929,944,-227,0,-113,-1062
5,249,118,12,133,5,15,132,9,1,5,124,-113,1,22,7,-113,14,118,-114,-100,3,9,236
6,842,-582,-103,708,3,-118,-332,-1055,-1043,-224,723,19,109,713,-235,-1161,846,-220,-231,698,-103,18,0
7,-100,264,21,-105,5,19,127,119,118,5,18,0,0,-100,136,138,-113,5,-118,19,9,1,0
8,-585,590,-115,473,132,1,475,19,13,-453,355,489,0,472,354,949,472,12,20,595,-571,-708,-708
9,118,-217,21,-109,-103,2,-205,-118,5,118,-223,-215,0,-223,-103,-236,-221,15,-113,-354,119,140,472


## 3.3) Mensagem descriptografada em números

In [7]:
decoded_bk = decoded_bk % 118

display(pd.DataFrame(decoded_bk))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,47,109,15,56,19,21,40,0,18,44,14,1,13,55,5,1,9,1,1,22,19,16,0
1,29,0,0,21,0,5,19,9,35,0,4,17,38,1,19,4,4,9,7,5,13,18,0
2,0,52,14,1,16,0,0,14,16,1,15,21,19,18,16,1,1,24,5,18,1,9,0
3,19,1,1,14,1,14,3,9,18,17,107,5,9,1,18,0,13,30,13,20,19,14,0
4,5,19,0,20,18,30,15,13,9,21,0,12,3,0,5,14,5,15,0,9,0,5,0
5,13,0,12,15,5,15,14,9,1,5,6,5,1,22,7,5,14,0,4,18,3,9,0
6,16,8,15,0,3,0,22,7,19,12,15,19,109,5,1,19,20,16,5,108,15,18,0
7,18,28,21,13,5,19,9,1,0,5,18,0,0,18,18,20,5,5,0,19,9,1,0
8,5,0,3,1,14,1,3,19,13,19,1,17,0,0,0,5,0,12,20,5,19,0,0
9,0,19,21,9,15,2,31,0,5,0,13,21,0,13,15,0,15,15,5,0,1,22,0


In [8]:
with open("conversion_table.json", encoding="utf-8") as f:
    conversion_tb = json.load(f)

print(conversion_tb)

{'0': ' ', '1': 'a', '2': 'b', '3': 'c', '4': 'd', '5': 'e', '6': 'f', '7': 'g', '8': 'h', '9': 'i', '10': 'j', '11': 'k', '12': 'l', '13': 'm', '14': 'n', '15': 'o', '16': 'p', '17': 'q', '18': 'r', '19': 's', '20': 't', '21': 'u', '22': 'v', '23': 'w', '24': 'x', '25': 'y', '26': 'z', '27': 'à', '28': 'â', '29': 'á', '30': 'ã', '31': 'ç', '32': 'é', '33': 'ê', '34': 'í', '35': 'ó', '36': 'ô', '37': 'õ', '38': 'ú', '39': 'ü', '40': 'A', '41': 'B', '42': 'C', '43': 'D', '44': 'E', '45': 'F', '46': 'G', '47': 'H', '48': 'I', '49': 'J', '50': 'K', '51': 'L', '52': 'M', '53': 'N', '54': 'O', '55': 'P', '56': 'Q', '57': 'R', '58': 'S', '59': 'T', '60': 'U', '61': 'V', '62': 'W', '63': 'X', '64': 'Y', '65': 'Z', '66': 'Á', '67': 'À', '68': 'Â', '69': 'Ã', '70': 'Ç', '71': 'É', '72': 'Ê', '73': 'Í', '74': 'Ó', '75': 'Ô', '76': 'Õ', '77': 'Ú', '78': 'Ü', '79': '0', '80': '1', '81': '2', '82': '3', '83': '4', '84': '5', '85': '6', '86': '7', '87': '8', '88': '9', '89': ':', '90': ';', '91': '<

In [9]:
decoded_ls = decoded_bk.flatten()

converted_ls = [conversion_tb[str(c)] for c in decoded_ls]

converted_bk = np.array(converted_ls).reshape(decoded_bk.shape)

display(pd.DataFrame(converted_bk))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,H,.,o,Q,s,u,A,,r,E,n,a,m,P,e,a,i,a,a,v,s,p,
1,á,,,u,,e,s,i,ó,,d,q,ú,a,s,d,d,i,g,e,m,r,
2,,M,n,a,p,,,n,p,a,o,u,s,r,p,a,a,x,e,r,a,i,
3,s,a,a,n,a,n,c,i,r,q,",",e,i,a,r,,m,ã,m,t,s,n,
4,e,s,,t,r,ã,o,m,i,u,,l,c,,e,n,e,o,,i,,e,
5,m,,l,o,e,o,n,i,a,e,f,e,a,v,g,e,n,,d,r,c,i,
6,p,h,o,,c,,v,g,s,l,o,s,.,e,a,s,t,p,e,-,o,r,
7,r,â,u,m,e,s,i,a,,e,r,,,r,r,t,e,e,,s,i,a,
8,e,,c,a,n,a,c,s,m,s,a,q,,,,e,,l,t,e,s,,
9,,s,u,i,o,b,ç,,e,,m,u,,m,o,,o,o,e,,a,v,


## 3.5) Mensagem descriptografada em caracteres alfanuméricos

In [10]:
converted_str = "".join(converted_bk.flatten('F'))

print(converted_str)

Há sempre alguma loucura no amor. Mas hâ sempre um pouco de razão na loucura.                   Quanto mais nos elevamos, menores parecenos aos olhos daqueles que não sabem voar.              As convicções são cârceres. Mais inimigas da verdade do que as próprias mentiras.               E aqueles que goram vistos dançando, foram julgados insanos por aqueles que não podiam escutar amúsica.                         Para ver muita coisa é preciso despregar os olhos de si mesmo. Nada neste mundo consome mais rapidamente o ser humano do que a paixão pelo ressentimento. A zantagem de ter péssima memória   divertir-se muitas vezes com as mesmas coisas boas como se fosse aprineira vez.                                 Friedrich Nietzche


In [11]:
encoded_ls = message_bk.flatten()

converted_ls = [conversion_tb[str(c)] for c in encoded_ls]

converted_bk = np.array(converted_ls).reshape(message_bk.shape)

display(pd.DataFrame(converted_bk))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,h,m,U,s,Ã,=,D,G,8,M,á,C,4,8,:,_,E,u,I,4,!,i,'
1,(,Ó,|,0,:,Q,f,X,6,Z,x,7,m,j,D,ê,(,8,2,S,.,$,U
2,s,',@,A,é,z,E,Q,E,z,Q,<,â,p,â,R,à,r,?,O,s,I,9
3,.,8,,,4,Í,ú,+,&,.,Ô,m,õ,Ú,é,í,d,f,M,K,),:,ç
4,y,g,a,[,Y,F,S,Õ,m,6,H,{,a,Ó,Õ,4,O,r,|,],Q,5,í
5,m,b,í,9,h,',í,H,L,ô,_,Ã,b,À,Í,B,m,',',#,],?,X
6,=,?,Z,',S,P,>,c,p,k,$,1,.,4,#,U,=,y,q,9,ç,z,2
7,l,J,7,u,2,Ô,L,ú,ó,M,a,7,.,f,Á,!,r,Ç,1,p,Z,à,)
8,Z,),Ü,Í,x,Y,i,O,1,ü,{,|,,Ô,u,F,0,',\,Ç,s,v,w
9,z,É,%,d,e,',_,ê,x,],X,<,,p,g,f,U,W,Â,ç,6,Ç,Ü


## 3.2) Mensagem a nosso encargo, em caracteres alfanuméricos:

In [12]:
converted_str = "".join(converted_bk.flatten("F"))

print(converted_str)

h(s.ym=lZz9a5sxDÁx$óV;sMÉÕ)IoH@êmÓ'8gb?J)Éõ1)ó>eKZÓ*ákTÓh5üVZÔhQU|@ aíZ7Ü%vw..                  s0A [9'uÍdakiEÀÊrÇà38%áRpTÚBQMúwÃ:é4YhS2xeÔ)V\ct- BC{ZIv]!wí=B'q=QzÍF'PÔY'Ü=môuvÍY              DfEúSí>Li_pÇE?7qmD#Vxg|Lj!Õs\2TâGXQ+ÕHcúOê)uVr<g6Zh6'PÚçF0:uXô!p86E&mLpó1x<l!o{p..              MZz.6ôkMü]oQ=dpécaaby}ÓÉ'6RôçH*éáxQÔH_$a{X;&6]çé)íêÕÓ=]úB'ÀõDJhrC7<m{Ã17|<iãA:uv3cêQHülf!4ÉC&tfa4mâõab..                        8jpÚÓÀ4fÔptFDÁCJ\'xng7AtüÍÀp<úxd:DâéÕÍ#Áugí{,UWló;YkP8oW*.e6ÓT3N_êRí4BU!FfS_ ú\jQÀwê[ÂKvXÚÊFSó(qE(àdOm=r0UyÍhLzKsnl/*LÓógUÉC6q!pu8rfr'yÇ'WT'ÜeZ;W0:}m{OÃÀ=_Ó.V%íI2?M|'q1\Â=bmÍÍ&}IwXM0uVCBVnÃm3m4SOK]#9pÇçwÃK?AY)-AêdYPuâÁ:s$õcr!.s)Q]çZs6]ífGcw*Âa,xYÃyâZõózyfai$I:5?zàvÇ.q..                  'U9çíX2)wÜA4_vÍ!'ÚqrpWSuFkHúhC7m


In [14]:
recoded_bk = np.matmul(key_bk, decoded_bk)

recoded_bk = recoded_bk % 118

display(pd.DataFrame(recoded_bk))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,8,13,60,19,69,92,43,46,87,52,29,42,83,87,89,114,44,21,48,83,96,9,97
1,103,74,116,79,89,56,6,63,85,65,24,86,13,10,43,33,103,87,81,58,109,99,60
2,19,102,95,40,32,26,44,56,44,26,56,91,28,16,28,57,27,18,94,54,19,48,88
3,109,87,0,0,83,73,38,106,101,109,75,13,37,77,32,34,4,6,52,50,104,89,31
4,25,7,1,111,64,45,58,76,13,85,47,115,1,74,76,83,54,18,116,113,56,84,34
5,13,2,34,88,8,102,34,47,51,36,114,69,2,67,73,41,13,97,102,98,113,94,63
6,92,94,65,102,58,55,93,3,16,11,99,80,109,83,98,60,92,25,17,88,31,26,81
7,12,49,86,21,81,75,51,38,35,52,1,86,109,6,66,96,18,70,80,16,65,27,104
8,65,104,78,73,24,64,9,54,80,39,115,116,0,75,21,45,79,102,112,70,19,22,23
9,26,71,100,4,5,102,114,33,24,113,63,91,0,16,7,6,60,62,68,31,85,70,78


In [16]:
compare_bk = np.equal(recoded_bk, message_bk)

display(pd.DataFrame(compare_bk).astype(int))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
5,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
7,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
8,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
9,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
